In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import chromadb
import os

load_dotenv()

True

In [3]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=1,
    groq_api_key=os.getenv("GROQ_API_KEY")  
)

In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.linkedin.com/jobs/view/4291320600/?alternateChannel=search&refId=f1FFN6UJ98sS9AQCAQYv1A%3D%3D&trackingId=XY1Rh6rhVHEPw8cuEull8A%3D%3D")
page_data = loader.load().pop().page_content

print(page_data)







 










WTMF AI hiring AI/ML Engineer in India | LinkedIn























 





      Skip to main content
    



LinkedIn







              AI/ML Engineer in Chennai
 
Expand search






This button displays the currently selected search type. When expanded it provides a list of search options that will switch the search inputs to match the current selection. 
 



                        Jobs
                    



                        People
                    



                        Learning
                    












Clear text






Clear text
















 





Clear text









 





Clear text















Clear text
















      Join now
    

          Sign in
      





 










AI/ML Engineer

WTMF AI
India



                Apply
      









AI/ML Engineer






                WTMF AI
              


              India
            



          

    
    
    
    
    
    
    
    
    
    
    
    